In [22]:
import time
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, col
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.functions import to_timestamp, date_format, hour, year, month, dayofmonth
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


In [23]:
sc

<SparkContext master=yarn appName=PySparkShell>

# Mongo Code

#### Schema of the Green Taxi Database - data was cleaned in mongo:

```
s3cmd get s3://spark-proj/data/yellow_tripdata_2017-05.csv s3://spark-proj/data/yellow_tripdata_2017-04.csv s3://spark-proj/data/yellow_tripdata_2017-03.csv s3://spark-proj/data/yellow_tripdata_2017-02.csv s3://spark-proj/data/yellow_tripdata_2017-01.csv s3://spark-proj/data/yellow_tripdata_2016-12.csv s3://spark-proj/data/yellow_tripdata_2016-11.csv s3://spark-proj/data/yellow_tripdata_2016-10.csv s3://spark-proj/data/yellow_tripdata_2016-09.csv s3://spark-proj/data/yellow_tripdata_2016-08.csv s3://spark-proj/data/green_tripdata_2017-05.csv s3://spark-proj/data/green_tripdata_2017-04.csv s3://spark-proj/data/green_tripdata_2017-03.csv s3://spark-proj/data/green_tripdata_2017-02.csv s3://spark-proj/data/green_tripdata_2017-01.csv s3://spark-proj/data/green_tripdata_2016-12.csv s3://spark-proj/data/green_tripdata_2016-11.csv s3://spark-proj/data/green_tripdata_2016-10.csv s3://spark-proj/data/green_tripdata_2016-09.csv s3://spark-proj/data/green_tripdata_2016-08.csv

mongoimport -d taxidb -c taxidata --type csv --headerline --file yellow_tripdata_2017-05.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file yellow_tripdata_2017-04.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file yellow_tripdata_2017-03.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file yellow_tripdata_2017-02.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file yellow_tripdata_2017-01.csv

mongoimport -d taxidb -c taxidata --type csv --headerline --file green_tripdata_2017-05.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file green_tripdata_2017-04.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file green_tripdata_2017-03.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file green_tripdata_2017-02.csv
mongoimport -d taxidb -c taxidata --type csv --headerline --file green_tripdata_2017-01.csv



```

### Import Green Taxi Data from Mongo Database ( 1M total)

Our data is currently stored in another EC2 database:

`mongodb://ec2-54-190-6-201.us-west-2.compute.amazonaws.com/taxidb_sm.taxidata_g`

In [24]:
start = time.time()
green_df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri","mongodb://ec2-54-190-6-201.us-west-2.compute.amazonaws.com/taxidb_sm.taxidata_g")\
    .load()
print time.time() - start

0.145874977112


In [25]:
green_df.printSchema()

root
 |-- DOLocationID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- color: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- extra: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- trip_type: integer (nullable = true)



### Weather Data ( per day per hour) - weather underground.com

In [26]:
start = time.time()
weather_df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://ec2-54-190-6-201.us-west-2.compute.amazonaws.com/taxidb.weather_data").load()
print time.time() - start

0.135233163834


In [27]:
def indexStringColumns(df, cols):
    newdf = df
    for c in cols:
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

In [28]:
w_df = indexStringColumns(weather_df, ['condition'])

In [29]:
w_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- date: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- time: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- condition: double (nullable = true)



### Yellow Taxi Data

In [30]:
start = time.time()
yellow_df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://ec2-54-190-6-201.us-west-2.compute.amazonaws.com/taxidb_sm.taxidata_y").load()
print time.time() - start

0.134047031403


In [31]:
yellow_df.printSchema()

root
 |-- DOLocationID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- color: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: double (nullable = true)



### Glue Yellow and Green Taxi data together for cleaning

In [32]:
select_cols = [x for x in yellow_df.columns if x!='_id' ]

y_df = yellow_df.select(select_cols)
y_df = y_df.sample(False,0.2, seed=1)
y_df.cache()

g_df = green_df.select(select_cols)
g_df = g_df.sample(False,0.1, seed=1)
g_df.cache()

total_df = y_df.unionAll(g_df)
total_df.cache()
total_df.show(2)

+------------+------------+----------+--------+-----+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|color|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|
+------------+------------+----------+--------+-----+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+
|         113|         158|         1|       1|  1.0|  0.5|       10.5|                  0.3|    0.5|              1|           2|                 N|       0.0|         0.0|        11.8|  2017-04-01 00:15:07| 2017-04-01 00:00:00|          1.8|
|         193|         2

### Feature Engineering - extracting data from dates

- Day of Week
- Day of Month
- Month
- Round to hour

In [33]:
def make_dates(df, colName, newCol):
    return df.withColumn(newCol,to_timestamp(colName, 'yyyy-MM-dd HH:mm:ss')).drop(colName)

def get_dateinfo(df, colName):
    return df.withColumn('dow', date_format(colName,'u').cast(IntegerType()))\
    .withColumn('hour', hour(colName))\
    .withColumn('day', dayofmonth(colName))\
    .withColumn('month', month(colName))\
    .withColumn('year', year(colName))
    
def indexStringColumns(df, cols):
    newdf = df
    for c in cols:
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

#### Clean the total dataset

In [34]:
start = time.time()

df_dates = make_dates(total_df, "tpep_dropoff_datetime", "dropoff_datetime")
df_dates = make_dates(df_dates, "tpep_pickup_datetime", "pickup_datetime")
new_df = indexStringColumns(df_dates, ["store_and_fwd_flag"])
new_df = get_dateinfo(df_dates, 'dropoff_datetime')
print time.time() - start

90.8021991253


### Join in the weather data

In [35]:
joined_df = new_df.join(w_df,[new_df.day==w_df.day, new_df.month==w_df.month, new_df.hour==w_df.hour], 'left_outer')
joined_df = joined_df.select(new_df.DOLocationID, new_df.PULocationID, new_df.RatecodeID, new_df.VendorID, new_df.color, new_df.extra, new_df.fare_amount, new_df.improvement_surcharge, new_df.mta_tax, new_df.passenger_count, new_df.payment_type, new_df.store_and_fwd_flag, new_df.tip_amount, new_df.tolls_amount, new_df.total_amount, new_df.trip_distance, new_df.dropoff_datetime, new_df.pickup_datetime, new_df.dow, new_df.hour, new_df.day, new_df.month, new_df.year, w_df.temp, w_df.condition)
joined_df = joined_df.drop("dropoff_datetime").drop("pickup_datetime").drop("store_and_fwd_flag").drop("ehail_fee").drop("trip_type")


### One-hot Encode any of the categorical Columns

In [36]:

start = time.time()

def oneHotEncodeColumns(df, cols):
    newdf = df
    for c in cols:
        #For each given colum, create OneHotEncoder. 
        #dropLast : Whether to drop the last category in the encoded vector (default: true)
        onehotenc = OneHotEncoder(inputCol=c, outputCol=c+"-onehot", dropLast=False)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-onehot" 
        #and then drops the original columns.
        #and drop the "-onehot" suffix. 
        newdf = onehotenc.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-onehot", c)
    return newdf

onehot_cols = [ x for x in joined_df.columns if 'ID' in x]
onehot_cols = onehot_cols + ['condition', 'dow','day','month']

joined_df.printSchema()
dfhot = oneHotEncodeColumns(joined_df,onehot_cols)
df_for_model = dfhot.withColumnRenamed("color","label")
df_for_model.printSchema()

print time.time() - start

root
 |-- DOLocationID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- VendorID: integer (nullable = true)
 |-- color: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- dow: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- condition: double (nullable = true)

root
 |-- label: double (nullable = true)
 |-- extra: double (nullable = true)
 |-

### Prepare the Vectors for Modeling

In [37]:
va = VectorAssembler(outputCol="features", inputCols=df_for_model.drop("label").columns) #except the last col.
taxi_points = va.transform(df_for_model).select("features", "label")


In [38]:
split_data = taxi_points.randomSplit([0.8, 0.2])
training = split_data[0].cache()
test = split_data[1].cache()

### Random Forest Modeling

- 3M : 217.947538137


In [39]:
start = time.time()
rf = RandomForestClassifier(maxDepth=20)
rfmodel = rf.fit(training)
print time.time() - start

217.947538137


In [40]:
start = time.time()
rfpredicts = rfmodel.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(rfpredicts)
print("Test Error = %g" % (1.0 - accuracy))
print time.time() - start

Test Error = 0.0477124
30.1611700058


#### Feature importance

In [42]:
import operator

columns = df_for_model.drop("label").columns
importances = {}
for score,col in zip(rfmodel.featureImportances, columns):
    importances[col] =  score
sorted_importances = sorted(importances.items(), key=operator.itemgetter(1), reverse = True)
sorted_importances

[('tip_amount', 0.020473327717663619),
 ('trip_distance', 0.015715295457191372),
 ('improvement_surcharge', 0.015558255022715064),
 ('fare_amount', 0.01521496550253601),
 ('total_amount', 0.015137444771896458),
 ('hour', 0.0099468248971165685),
 ('passenger_count', 0.0084854350164258272),
 ('month', 0.0052412768762240877),
 ('payment_type', 0.0039374205747062709),
 ('extra', 0.003339054355755852),
 ('temp', 0.0028635818705883618),
 ('tolls_amount', 0.0013714347684928045),
 ('mta_tax', 0.0011805397472915171),
 ('condition', 4.3180992096275849e-05),
 ('PULocationID', 2.3114616875778983e-05),
 ('VendorID', 3.3505529096956143e-06),
 ('day', 1.2891682775805025e-06),
 ('DOLocationID', 0.0),
 ('year', 0.0),
 ('RatecodeID', 0.0),
 ('dow', 0.0)]

### Logistic Regression

In [ ]:
start = time.time()
logreg = LogisticRegression()
log_model = logreg.fit(training)
print time.time() - start


In [ ]:
start = time.time()
logpredicts = log_model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(logpredicts)
print("Test Error = %g" % (1.0 - accuracy))
print time.time() - start

### Feature Importance